# extract SP500 historical constituents from Wikipedia

Reference: https://analyzingalpha.com/sp500-historical-components-and-changes



In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')


data contains 2 tables: current members and historical changes

In [3]:
# 1st table is current members
data[0].head(3)

,0,1,2,3,4,5,6,7,8
0,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
1,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
2,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888


In [4]:
# columns 0,1,6,7 are useful - trim off first row that contains headings - make our own
# fix null dates - give them 'impossible' value so they show up in date range queries

sp500 = data[0].loc[1:,[0,1,6]]
columns = ['ticker', 'name', 'date']
sp500.columns = columns
sp500.loc[sp500['date'].isnull(), 'date'] = '1957-01-01'


In [5]:
# bad format
sp500.query('date.str.len() != "10"')

,ticker,name,date
55,T,AT&T Inc.,1983-11-30 (1957-03-04)


In [6]:
# use a regex -- need to do some googling
# https://www.regular-expressions.info/numericranges.html
sp500[sp500.date.str.match('^[1-2][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]$') != True]

# ATT is bad

,ticker,name,date
55,T,AT&T Inc.,1983-11-30 (1957-03-04)


In [7]:
# look in original data
data[0][ (data[0][6].str.len() != 10) & (~data[0][6].isnull())]


,0,1,2,3,4,5,6,7,8
0,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
55,T,AT&T Inc.,reports,Communication Services,Integrated Telecommunication Services,"Dallas, Texas",1983-11-30 (1957-03-04),0000732717,1983 (1885)


In [8]:
# fix it
sp500.loc[~sp500.date.str.match('^[1-2][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]$'),'date'] = '1983-11-30'

# check
sp500.query('ticker == "T"')

 

,ticker,name,date
55,T,AT&T Inc.,1983-11-30


In [9]:
# convert string to datetime type
# https://stackoverflow.com/questions/34962104/pandas-how-can-i-use-the-apply-function-for-a-single-column

sp500['dt'] = sp500['date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [10]:
sp500['date'].at(0)[6]

'2015-08-31'

In [11]:
# add an action variable - not sure if we need it but could be useful
sp500['action'] = 'add'


In [12]:

# are there dups?
print('No dups? {}'.format(len(sp500) == len(sp500.ticker.unique())))


No dups? True


In [13]:

# pickle it
import utils
utils.pickle_file(sp500, 'sp500-current.p')

In [14]:
# now deal with the historical changes

changes = data[1]

changes[2:]   # messy

# looking at the wiki article we see that column 1 are adds, column 3 are removes; both for the date 
# in column 0. I'll flatten this out usng a seperate rows and an action code

# sp500 = data[0].loc[1:,[0,1,6,7]]
# columns = ['added_ticker', 'name', 'date', 'cik']


sp500_adds = changes.loc[2:,[0,1]]
columns = ['date', 'ticker']
sp500_adds.columns = columns
sp500_adds['action'] = 'add'

# append the remove actions
tmp = changes.loc[2:,[0,3]]
tmp.columns = columns
tmp['action'] = 'rem'
sp500_adds = sp500_adds.append(tmp)

# drop null ticker values

sp500_adds = sp500_adds[~sp500_adds.ticker.isnull()]
sp500_adds['dt'] = sp500_adds['date'].apply(lambda x: datetime.strptime(x,'%B %d, %Y'))
sz1 = len(sp500_adds)
sp500_adds.drop_duplicates(keep=False,inplace=True) 
drops = sz1 != len(sp500_adds)

print ('num records {}'.format(sz1))

if drops:
    print('you dropped something')


#print(sp500_adds.head(5))
#print(sp500_adds.tail(5))

# make sure all the datetimes are valid
sp500_adds[sp500_adds.dt.isnull()]

right_now = datetime.now()

print(type(right_now))
print(sp500_adds.dtypes)
df = sp500_adds

# test how query functions work
df[(df['dt'] > '1924-1-1') & (df['action'] == 'add')]

now = pd.datetime.today()
print(df.query("dt > @right_now  "))


num records 475
<class 'datetime.datetime'>
date              object
ticker            object
action            object
dt        datetime64[ns]
dtype: object
Empty DataFrame
Columns: [date, ticker, action, dt]
Index: []


In [15]:
# save it
utils.pickle_file(sp500_adds, 'sp500-deltas.p')

In [22]:
# come up with a routine to get the components for any given date

import utils as u
from collections import Counter

class SP500Components:
    # static vars - fyi initiailized when first encountered in code 
    _sp500_current_df = u.read_pickle('sp500-current.p')
    _sp500_deltas_df = u.read_pickle('sp500-deltas.p')
    # cache current members as Counter 
    # - allowing duplicate values and graceful handling 
    # of possible multiple add/drop for same ticker
    _sp500_current = Counter(_sp500_current_df['ticker'].tolist())

    def __init__(self):
        pass
        
        
    def from_date(self, datetime):
        # basic algo is
        # S(t) = S(current) + S(drops > date) - S(add > date) 
        # df = self._sp500_current_df
        # curr = Counter(df[df['dt'] < datetime]['ticker'].tolist())
        # print('num curr < datetime {} is {}'.format(datetime,len(curr)))
        adds = Counter(SP500Components._sp500_deltas_df
                       .query('action == "add" and dt <= @datetime')['ticker'].tolist())
        #print('adds {}'.format(adds))
        drops = Counter(SP500Components._sp500_deltas_df
                        .query('action == "rem" and dt <= @datetime')['ticker'].tolist())
        #print('drops {}'.format(drops))
        ret = SP500Components._sp500_current + drops - adds
        print('diff ret - current {}'.format(ret - self._sp500_current))
        print('diff current - ret {}'.format(self._sp500_current - ret))
        return list(ret)


comps = SP500Components()

now = datetime.strptime('2020-01-01','%Y-%d-%m').date()



now2 = datetime.strptime('2010-01-04','%Y-%d-%m').date()

print('components:')
elems = comps.from_date(now)
elems2 = comps.from_date(now2)

print('diff1 {}'.format(set(elems) - set(elems2)))

print('diff2{}'.format(set(elems2) - set(elems)))

print('num elems {}'.format(len(elems)))
print('num elems2 {}'.format(len(elems2)))


components:
diff ret - current Counter({'CELG': 1, 'WIN': 1, 'DJ': 1, 'WB': 1, 'S': 1, 'STI': 1, 'SVU': 1, 'PCL': 1, 'LIFE': 1, 'TE': 1, 'OI': 1, 'BJS': 1, 'RHT': 1, 'SPLS': 1, 'ITT': 1, 'BCR': 1, 'TWX': 1, 'RAI': 1, 'LXK': 1, 'JBL': 1, 'FLR': 1, 'DPS': 1, 'POM': 1, 'GAS': 1, 'GHC': 1, 'PCS': 1, 'FDO': 1, 'NSM': 1, 'GR': 1, 'MWW': 1, 'BS': 1, 'RRD': 1, 'CVH': 1, 'PCG': 1, 'MJN': 1, 'JNY': 1, 'FRX': 1, 'TGNA': 1, 'GENZ': 1, 'HSP': 1, 'DD': 1, 'KSE': 1, 'SAI': 1, 'MIL': 1, 'BEAM': 1, 'BRCM': 1, 'SHLD': 1, 'DTV': 1, 'SLE': 1, 'PBI': 1, 'EQT': 1, 'WFM': 1, 'MUR': 1, 'RDC': 1, 'FRE': 1, 'CSC': 1, 'VIAB': 1, 'R': 1, 'ODP': 1, 'BMC': 1, 'APC': 1, 'THC': 1, 'SCG': 1, 'JCP': 1, 'AET': 1, 'TWC': 1, 'DNR': 1, 'SRCL': 1, 'AKS': 1, 'LEH': 1, 'X': 1, 'RSH': 1, 'JCI': 1, 'BIG': 1, 'TIE': 1, 'ACAS': 1, 'CBE': 1, 'TSG': 1, 'HNZ': 1, 'NE': 1, 'NYX': 1, 'TER': 1, 'CNX': 1, 'CFN': 1, 'CCE': 1, 'AN': 1, 'URBN': 1, 'MMI': 1, 'AV': 1, 'MAT': 1, 'GRA': 1, 'TLAB': 1, 'JEF': 1, 'XTO': 1, 'COL': 1, 'LM': 1, 'PTV

In [23]:
elems2.index('BRK.B')
#df = u.read_pickle('sp500-current')

#len(df[df['dt'] < '2020-1-1'])

456